<a href="https://colab.research.google.com/github/robinjameslee/Deribit-Covered-Call-Strategy/blob/main/Deribit_Covered_Call_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script aims to have a look at the Covered Call Strategy Yield regarding  futures listed on Deribit.

For a covered call, you buy the underlying then sell a call option
Using the premium you receive from the call option, you will limit your upside gain (if the underlying rises above the strike price), but you'll also protect your downside risk if the underlying price tanks.

Data are queried using Deribit's API: [docs.deribit.com](https://docs.deribit.com)

In [22]:
import json
import requests
import pandas as pd
import datetime
import numpy as np

pd.set_option('display.float_format', '{:.10f}'.format)

def get_instruments(currency, kind, expired='false'):
  url = 'https://www.deribit.com/api/v2/public/get_instruments?'
  parameters = {'currency': currency, 'kind': kind, 'expired': expired}
  res = requests.get(url, params=parameters)
  res_dict = json.loads(res.content)['result']
  return res_dict

def get_spot(underlying):
  url = 'https://www.deribit.com/api/v2/public/get_index_price?'
  res = requests.get(url, params={'index_name': underlying})
  spot_price = json.loads(res.content)['result']['index_price']
  return spot_price

def get_book_summary_by_currency(currency, kind):
  url = 'https://www.deribit.com/api/v2/public/get_book_summary_by_currency?'
  parameters = {'currency': currency, 'kind': kind}
  res = requests.get(url, params=parameters)
  res_dict = json.loads(res.content)['result']
  return res_dict

underlying = 'ETH' #BTC or ETH
underlying_spot = get_spot(f'{underlying.lower()}_usdc')
underlying_options = get_instruments(underlying, 'option')
underlying_options_book_summary = pd.DataFrame(get_book_summary_by_currency(underlying, 'option'))

In [23]:
filtered_call_option_df = pd.DataFrame([], columns=['instrument_name', 'expiry', 'strike'])
today_plus_30 = datetime.datetime.today() + datetime.timedelta(days=30)

#filter the ATM and slightly OTM call options
for options in underlying_options:
  expiry = pd.to_datetime(options['expiration_timestamp'], unit='ms')
  strike = options['strike']
  if options['option_type'] == 'call' and expiry >= today_plus_30 and strike >= underlying_spot * 0.95 and strike <= underlying_spot * 1.2:
    filtered_call_option_df.loc[len(filtered_call_option_df)] = [options['instrument_name'], expiry, strike]

filtered_call_option_df.head()

,instrument_name,expiry,strike
0,ETH-30AUG24-3300-C,2024-08-30 08:00:00,3300.0000000000
1,ETH-30AUG24-3400-C,2024-08-30 08:00:00,3400.0000000000
2,ETH-30AUG24-3500-C,2024-08-30 08:00:00,3500.0000000000
3,ETH-30AUG24-3600-C,2024-08-30 08:00:00,3600.0000000000
4,ETH-30AUG24-3700-C,2024-08-30 08:00:00,3700.0000000000


In [24]:
#Merge both dataframes and calculate our pnl under different scenarios
underlying_options_book_summary = underlying_options_book_summary[['instrument_name', 'underlying_price', 'mark_iv', 'mark_price', 'bid_price', 'ask_price']]
df = filtered_call_option_df.merge(underlying_options_book_summary, on='instrument_name')
df['day_to_expiry'] = df['expiry'] - datetime.datetime.today()

df['time_value_premium'] = df['bid_price'] - np.where(underlying_spot > df['strike'], underlying_spot - df['strike'], 0) / underlying_spot
df['time_value_premium_annualized_pct'] = (1 + df['time_value_premium']) ** (365 / df['day_to_expiry'].dt.days) - 1

df['pnl_if_spot_minus_10_pct'] = df['bid_price'] - 0.1
df['pnl_if_spot_minus_30_pct'] = df['bid_price'] - 0.3

df['risk_reward_spot_minus_10_pct'] = df['time_value_premium'] / (1 + df['pnl_if_spot_minus_10_pct'])
df['risk_reward_spot_minus_30_pct'] = df['time_value_premium'] / (1 + df['pnl_if_spot_minus_30_pct'])

df

,instrument_name,expiry,strike,underlying_price,mark_iv,mark_price,bid_price,ask_price,day_to_expiry,time_value_premium,time_value_premium_annualized_pct,pnl_if_spot_minus_10_pct,pnl_if_spot_minus_30_pct,risk_reward_spot_minus_10_pct,risk_reward_spot_minus_30_pct
0,ETH-30AUG24-3300-C,2024-08-30 08:00:00,3300.0000000000,3490.9300000000,66.5300000000,0.1140820000,0.1125000000,0.1155000000,38 days 17:00:56.899863,0.0691741137,0.9011564859,0.0125000000,-0.1875000000,0.0683201123,0.0851373708
1,ETH-30AUG24-3400-C,2024-08-30 08:00:00,3400.0000000000,3490.9300000000,66.5900000000,0.0988980000,0.0980000000,0.0995000000,38 days 17:00:56.899863,0.0836642384,1.1635810853,-0.0020000000,-0.2020000000,0.0838319022,0.1048424040
2,ETH-30AUG24-3500-C,2024-08-30 08:00:00,3500.0000000000,3490.9300000000,66.7200000000,0.0853520000,0.0845000000,0.0860000000,38 days 17:00:56.899863,0.0845000000,1.1796620344,-0.0155000000,-0.2155000000,0.0858303707,0.1077119184
3,ETH-30AUG24-3600-C,2024-08-30 08:00:00,3600.0000000000,3490.9300000000,67.1100000000,0.0736470000,0.0730000000,0.0745000000,38 days 17:00:56.899863,0.0730000000,0.9675164951,-0.0270000000,-0.2270000000,0.0750256937,0.0944372574
4,ETH-30AUG24-3700-C,2024-08-30 08:00:00,3700.0000000000,3490.9300000000,67.5500000000,0.0634010000,0.0630000000,0.0640000000,38 days 17:00:56.899863,0.0630000000,0.7982869315,-0.0370000000,-0.2370000000,0.0654205607,0.0825688073
5,ETH-30AUG24-3800-C,2024-08-30 08:00:00,3800.0000000000,3490.9300000000,68.0700000000,0.0545390000,0.0540000000,0.0550000000,38 days 17:00:56.899863,0.0540000000,0.6572578245,-0.0460000000,-0.2460000000,0.0566037736,0.0716180371
6,ETH-30AUG24-3900-C,2024-08-30 08:00:00,3900.0000000000,3490.9300000000,68.6300000000,0.0468990000,0.0465000000,0.0475000000,38 days 17:00:56.899863,0.0465000000,0.5473926312,-0.0535000000,-0.2535000000,0.0491283677,0.0622906899
7,ETH-30AUG24-4000-C,2024-08-30 08:00:00,4000.0000000000,3490.9300000000,69.2700000000,0.0403700000,0.0400000000,0.0410000000,38 days 17:00:56.899863,0.0400000000,0.4575038359,-0.0600000000,-0.2600000000,0.0425531915,0.0540540541
8,ETH-30AUG24-4100-C,2024-08-30 08:00:00,4100.0000000000,3490.9300000000,69.9600000000,0.0348140000,0.0340000000,0.0355000000,38 days 17:00:56.899863,0.0340000000,0.3787121281,-0.0660000000,-0.2660000000,0.0364025696,0.0463215259
9,ETH-27SEP24-3300-C,2024-09-27 08:00:00,3300.0000000000,3519.7700000000,68.6800000000,0.1470290000,0.1455000000,0.1475000000,66 days 17:00:56.899863,0.1021741137,0.7126019979,0.0455000000,-0.1545000000,0.0977275120,0.1208446053
